In [1]:
##import packages used in this session
import pandas as pd
from random import sample

In [2]:
pwd

'/Users/mkbelay/Documents/PPOL 564/Data Science Project/PPOL-564-Data-Science-Project/Data Analysis'

## Data collection and cleaning

### COVID-19 cases by county, race, and ethnicity
This data source was obtained from the New York Times public GitHub repository. The unit of observation is state; this data frame reports the number of reported COVID-19 cases by race and ethnicity for counties that provided this information to the CDC. The timeframe is from the beginning of the pandemic until the end of May 2020. https://github.com/nytimes/covid-19-data/issues/381

In [3]:
##import NYT data 
county_data=pd.read_csv("data.csv",dtype={'fips': object})

In [4]:
county_data.shape

(974, 23)

There are 974 counties represented in this data. There are approximately 3,007 counties in the United States. 

In [5]:
## column headers
county_data.columns

Index(['fips', 'state', 'county', 'cases', 'white_cases', 'black_cases',
       'hispanic_cases', 'native_cases', 'asian_cases', 'white_rate',
       'black_rate', 'hispanic_rate', 'native_rate', 'asian_rate', 'known_pct',
       'pop_white', 'pop_black', 'pop_hispanic', 'pct_white', 'pct_black',
       'pct_hispanic', 'pct_asian', 'pct_native'],
      dtype='object')

The NYT data set contains pre-calculated COVID-19 case rates amongst different racial and ethnic populations and also contains population data for white, black, and hispanic populations in each of the 974 counties.Note the population data for asian and native american populations is not included. Since the outcome variable for the statistical learning componenet will rely on population data for all minority populations represented in this data (black, hispanic, native american, and asian), only the COVID-19 cases will remain in this data set, and population and COVID-19 rates for each racial/ethnic category in each county will be calculated later on using the most recent population data from the U.S. Census. 

In [6]:
## keep relevant columsn(i.e., county fips code; state; county; total cases; cases by race)
county_data=county_data.filter(items=['fips', 'state', 'county', 'cases', 'white_cases', 'black_cases',
       'hispanic_cases', 'native_cases', 'asian_cases'])

In [7]:
county_data.head(10)

,fips,state,county,cases,white_cases,black_cases,hispanic_cases,native_cases,asian_cases
0,01001,Alabama,Autauga County,119,44,21,0,0,0
1,01003,Alabama,Baldwin County,201,144,6,0,0,0
2,01005,Alabama,Barbour County,91,5,32,0,0,0
3,01007,Alabama,Bibb County,50,17,5,0,0,0
4,01009,Alabama,Blount County,37,15,0,0,0,0
5,01011,Alabama,Bullock County,125,0,106,0,0,0
6,01013,Alabama,Butler County,263,65,166,0,0,0
7,01015,Alabama,Calhoun County,124,72,23,0,0,0
8,01017,Alabama,Chambers County,301,81,181,0,0,0
9,01021,Alabama,Chilton County,65,27,0,0,0,0


For ease of analysis and visualization later on, the state names will be converted to two-letter abbreviations. A publicly available dictionary, created and made available by Roger Allen (https://gist.github.com/rogerallen/1583593), will be used for this conversion.

In [8]:
##use code provided by roger allen
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [9]:
##convert state names to two-letter abbreviations
county_data['state'] = county_data['state'].apply(us_state_abbrev.get)

In [10]:
##view dataframe; confirm changes.
county_data.sample(20)

,fips,state,county,cases,white_cases,black_cases,hispanic_cases,native_cases,asian_cases
517,28043,MS,Grenada County,63,27,5,0,0,0
659,37037,NC,Chatham County,585,121,24,175,0,0
39,01097,AL,Mobile County,1236,369,632,0,0,0
447,26139,MI,Ottawa County,705,284,0,245,0,0
210,13141,GA,Hancock County,88,6,19,0,0,0
855,45079,SC,Richland County,1376,274,677,45,0,0
891,51089,VA,Henry County,80,11,5,0,0,0
885,51067,VA,Franklin County,17,10,0,0,0,0
278,17143,IL,Peoria County,156,45,42,0,0,0
899,51137,VA,Orange County,77,0,0,5,0,0


In [11]:
##remove leading zeros from fips code
county_data['fips'] = [ i.lstrip('0') for i in county_data['fips'] ]

In [12]:
## check dtypes
county_data.dtypes

fips              object
state             object
county            object
cases              int64
white_cases        int64
black_cases        int64
hispanic_cases     int64
native_cases       int64
asian_cases        int64
dtype: object

### Population: https://www.census.gov/newsroom/press-kits/2020/population-estimates-detailed.html
NYT missing population data for Native American and Asian population.

In [13]:
##population data
county_pop=pd.read_csv("cc-est2019-alldata.csv", encoding='ISO-8859-1', dtype={'COUNTY': object,'STATE': object})

In [14]:
## based on the code book,  2019 is coded as 12 under column YEAR
county_pop=county_pop.loc[county_pop['YEAR'] == 12]

In [15]:
## population estimate for all ages is coded as 0 under column AGEGRP
county_pop=county_pop.loc[county_pop['AGEGRP'] == 0]

In [16]:
## keep only relevant columns
county_pop=county_pop.filter(items=['STATE','COUNTY', 'STNAME', 'CTYNAME','TOT_POP','H_MALE', 'H_FEMALE',
                                    'WA_MALE', 'WA_FEMALE', 'BA_MALE', 'BA_FEMALE', 'IA_MALE', 'IA_FEMALE', 
                                    'AA_MALE', 'AA_FEMALE', 'NA_MALE', 'NA_FEMALE','TOM_MALE', 'TOM_FEMALE' ])

In [17]:
county_pop

,STATE,COUNTY,STNAME,CTYNAME,TOT_POP,H_MALE,H_FEMALE,WA_MALE,WA_FEMALE,BA_MALE,BA_FEMALE,IA_MALE,IA_FEMALE,AA_MALE,AA_FEMALE,NA_MALE,NA_FEMALE,TOM_MALE,TOM_FEMALE
209,01,001,Alabama,Autauga County,55869,884,787,20878,21729,5237,6000,121,145,286,370,32,26,538,507
437,01,003,Alabama,Baldwin County,223234,5545,4989,94810,100388,9486,10107,903,839,932,1448,74,80,2042,2125
665,01,005,Alabama,Barbour County,24686,629,488,6389,5745,6311,5595,103,67,55,61,34,18,172,136
893,01,007,Alabama,Bibb County,22394,343,280,8766,8425,2941,1822,53,50,23,25,22,4,124,139
1121,01,009,Alabama,Blount County,57826,2950,2632,27258,28154,516,462,192,178,85,100,42,25,379,435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715445,56,037,Wyoming,Sweetwater County,42343,3551,3221,20446,19252,347,251,324,302,203,250,35,34,453,446
715673,56,039,Wyoming,Teton County,23464,1884,1670,11567,10718,101,71,106,102,143,252,20,13,205,166
715901,56,041,Wyoming,Uinta County,20226,927,944,9753,9524,77,75,135,157,38,62,17,14,204,170
716129,56,043,Wyoming,Washakie County,7805,565,543,3759,3618,25,19,63,75,25,39,4,2,87,89


In [18]:
##combine state and county fips to make consistent with NYT dataframe
county_pop['fips'] = county_pop[['STATE', 'COUNTY']].apply(lambda x: ''.join(x), axis=1)
##remove leading zeros from fips codes
county_pop['fips'] = [ i.lstrip('0') for i in county_pop['fips'] ]

In [19]:
## convert state names to two-letter abbreviations
##county_pop['STNAME'] = county_pop['STNAME'].apply(us_state_abbrev.get)

In [20]:
##sum male and female population to get total population for all races and ethnicities 
county_pop['WH']=county_pop['WA_MALE']+county_pop['WA_FEMALE']
county_pop['BA']=county_pop['BA_MALE']+county_pop['BA_FEMALE']
county_pop['AA']=county_pop['AA_MALE']+county_pop['AA_FEMALE']
county_pop['IA']=county_pop['IA_MALE']+county_pop['IA_FEMALE']
county_pop['HI']=county_pop['H_MALE']+county_pop['H_FEMALE']

In [21]:
##keep relevant columns
county_pop=county_pop.filter(items=['fips','TOT_POP','WH','BA','AA','IA','HI'])

In [22]:
## rename state and county name columns
##county_pop=county_pop.rename(columns={'STNAME':'state','CTYNAME':'county'})

In [23]:
##view data frame.
county_pop

,fips,TOT_POP,WH,BA,AA,IA,HI
209,1001,55869,42607,11237,656,266,1671
437,1003,223234,195198,19593,2380,1742,10534
665,1005,24686,12134,11906,116,170,1117
893,1007,22394,17191,4763,48,103,623
1121,1009,57826,55412,978,185,370,5582
...,...,...,...,...,...,...,...
715445,56037,42343,39698,598,453,626,6772
715673,56039,23464,22285,172,395,208,3554
715901,56041,20226,19277,152,100,292,1871
716129,56043,7805,7377,44,64,138,1108


### Economic indicators

#### County GDP
County level GDP data is obtained from the Bureau of Economic Analysis at the U.S. Department of Commerce. The metric selected is GDP by county and metropolitan area across all industries. This data was last updated on December 12, 2019.
https://apps.bea.gov/iTable/iTable.cfm?reqid=70&step=1&isuri=1&acrdn=5#reqid=70&step=1&isuri=1&acrdn=5

In [24]:
gdp=pd.read_csv("GDP_county - GDP_county.csv")

In [25]:
gdp.tail(11)

,GeoFips,GeoName,2018
3115,56045,"Weston, WY",318545
3116,Legend / Footnotes:,NaN,NaN
3117,1/ Gross Domestic Product (GDP) is in thousand...,NaN,NaN
3118,"* Broomfield County, CO, was created from part...",NaN,NaN
3119,* Estimates from 2008 forward separate Skagway...,NaN,NaN
3120,* Virginia combination areas consist of one or...,NaN,NaN
3121,"* Shannon County, SD was renamed to Oglala Lak...",NaN,NaN
3122,"* Kalawao County, Hawaii is combined with Maui...",NaN,NaN
3123,Metropolitan Areas are defined (geographically...,NaN,NaN
3124,(NA) Not available.,NaN,NaN


Tail of the data frame includes irrelevant rows that needs to be removed before merging this dataframe with the county level COVID-19 cases dataframe.

In [26]:
##remove irrelevant rows
gdp.drop(gdp.tail(10).index, inplace = True) 

In [27]:
## confirm removal
gdp.tail(11)

,GeoFips,GeoName,2018
3105,56025,"Natrona, WY",5672135
3106,56027,"Niobrara, WY",130556
3107,56029,"Park, WY",1460391
3108,56031,"Platte, WY",577915
3109,56033,"Sheridan, WY",1397518
3110,56035,"Sublette, WY",1245979
3111,56037,"Sweetwater, WY",3880016
3112,56039,"Teton, WY",2505534
3113,56041,"Uinta, WY",927537
3114,56043,"Washakie, WY",379984


The column headers need to be renamed to maintain consistency across all data frames that will be merged later on. 

In [28]:
##rename column headers
gdp = gdp.rename(columns={'GeoFips':'fips','2018': 'GDP'})    

In [29]:
##keep only relevant columns
gdp=gdp.filter(items=['fips','GDP'])

In [30]:
gdp

,fips,GDP
0,1001,1690937
1,1003,6606080
2,1005,851956
3,1007,424510
4,1009,942904
...,...,...
3111,56037,3880016
3112,56039,2505534
3113,56041,927537
3114,56043,379984


In [31]:
##Unlike the NYT data set, the county and state names are in a single column. 
##This will be remedied by creating a separate column for states.
##The county column will have the word 'county' added to all observations to maintain c
##onsistency with the NYT data set.

In [32]:
## create a separate state column
##gdp['state']=gdp['county'].str.split(",").str[1]

In [33]:
##retain only county names in the county column
#gdp['county']= gdp['county'].str.split(",").str[0].astype(str)

Unlike the NYT dataframe, the county column in this dataframe does not have the word county. To remedy this, first check if any of the observations in the NYT dataframe do not have the word county. 

In [34]:
##query counties in NYT dataframe without the word county
#county_exclude=county_data[~county_data['county'].str.contains(r'County')]
#county_exclude=county_exclude['county']
#county_exclude

In [35]:
## add 'county' to  observations in the county column, excluding the ones queried above
##gdp['county'] = gdp['county']+' County'  

In [36]:
## view data frame; confirm changes
gdp

,fips,GDP
0,1001,1690937
1,1003,6606080
2,1005,851956
3,1007,424510
4,1009,942904
...,...,...
3111,56037,3880016
3112,56039,2505534
3113,56041,927537
3114,56043,379984


#### County poverty and median household income
Poverty (poverty estimate for all ages and percent in poverty for all ages) and median household income by county was obtained from  U.S. Census Bureau, Small Area Income and Poverty Estimates (SAIPE) Program (i.e.,2018 Poverty and Median Household Income Estimates). This data was last revised on December 2019.
https://www.census.gov/data/datasets/2018/demo/saipe/2018-state-and-county.html

In [37]:
##import data;read FIPS code columns as dtype object to maintain traiiling zeroes.
pov_inc=pd.read_csv("est18all.xlsx - est18ALL.csv",skiprows=1, dtype={'County FIPS Code': object,'State FIPS Code': object})

In [38]:
##examine shape of data
pov_inc.shape

(3194, 31)

In [39]:
##view random sample of dataframe to understand the distribution of the data
pov_inc.sample(10)

,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Estimate, All Ages",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, All Ages",90% CI Lower Bound.1,90% CI Upper Bound.1,...,90% CI Upper Bound.5,Median Household Income,90% CI Lower Bound.6,90% CI Upper Bound.6,"Poverty Estimate, Age 0-4",90% CI Lower Bound.7,90% CI Upper Bound.7,"Poverty Percent, Age 0-4",90% CI Lower Bound.8,90% CI Upper Bound.8
2315,42,063,PA,Indiana County,"12,356","10,284","14,428",15.5,12.9,18.1,...,21.7,"47,999","44,330","51,668",.,.,.,.,.,.
191,06,000,CA,California,"4,972,955","4,927,111","5,018,799",12.8,12.7,12.9,...,17.3,"75,250","74,934","75,566","416,678","403,143","430,213",17.5,16.9,18.1
2457,46,105,SD,Perkins County,394,306,482,13.7,10.6,16.8,...,21.1,"55,626","51,917","59,335",.,.,.,.,.,.
1289,26,069,MI,Iosco County,"3,724","3,018","4,430",15.1,12.2,18.0,...,34.5,"41,256","38,986","43,526",.,.,.,.,.,.
1536,29,053,MO,Cooper County,"2,242","1,811","2,673",14.0,11.3,16.7,...,22.6,"52,513","49,221","55,805",.,.,.,.,.,.
2635,48,135,TX,Ector County,"20,485","16,336","24,634",12.9,10.3,15.5,...,22.1,"60,431","56,099","64,763",.,.,.,.,.,.
803,18,181,IN,White County,"2,025","1,538","2,512",8.5,6.4,10.6,...,16.8,"58,380","55,894","60,866",.,.,.,.,.,.
34,01,065,AL,Hale County,"3,705","2,931","4,479",25.6,20.3,30.9,...,44.5,"34,724","30,954","38,494",.,.,.,.,.,.
505,13,217,GA,Newton County,"15,946","13,270","18,622",14.8,12.3,17.3,...,27.0,"56,598","52,162","61,034",.,.,.,.,.,.
294,08,085,CO,Montrose County,"4,854","3,868","5,840",11.6,9.2,14.0,...,22.1,"52,576","48,233","56,919",.,.,.,.,.,.


In [40]:
## examine column headers
pov_inc.columns

Index(['State FIPS Code', 'County FIPS Code', 'Postal Code', 'Name',
       'Poverty Estimate, All Ages', '90% CI Lower Bound',
       '90% CI Upper Bound', 'Poverty Percent, All Ages',
       '90% CI Lower Bound.1', '90% CI Upper Bound.1',
       'Poverty Estimate, Age 0-17', '90% CI Lower Bound.2',
       '90% CI Upper Bound.2', 'Poverty Percent, Age 0-17',
       '90% CI Lower Bound.3', '90% CI Upper Bound.3',
       'Poverty Estimate, Age 5-17 in Families', '90% CI Lower Bound.4',
       '90% CI Upper Bound.4', 'Poverty Percent, Age 5-17 in Families',
       '90% CI Lower Bound.5', '90% CI Upper Bound.5',
       'Median Household Income', '90% CI Lower Bound.6',
       '90% CI Upper Bound.6', 'Poverty Estimate, Age 0-4',
       '90% CI Lower Bound.7', '90% CI Upper Bound.7',
       'Poverty Percent, Age 0-4', '90% CI Lower Bound.8',
       '90% CI Upper Bound.8'],
      dtype='object')

For the purpose of this analysis, only the 'Poverty Percent,All ages' and the 'Median Household Income' variables will be used.

In [41]:
##keep only relevant columns
pov_inc=pov_inc.filter(items=['State FIPS Code','County FIPS Code','Poverty Percent, All Ages','Median Household Income'])

The format of the county fips code in this data is shortened when compared to the above data frame. The state fips code and county fips code will be combined below to create a consistent fips code. 

In [42]:
##join county and state fips codes
pov_inc['fips'] = pov_inc[['State FIPS Code', 'County FIPS Code']].apply(lambda x: ''.join(x), axis=1)
##remove leading zeros from fips codes
pov_inc['fips'] = [ i.lstrip('0') for i in pov_inc['fips'] ]

In [43]:
##view dataframe to confirm creation of new column
pov_inc.sample(10)

,State FIPS Code,County FIPS Code,"Poverty Percent, All Ages",Median Household Income,fips
1982,37,115,17.0,"44,693",37115
85,02,164,18.2,"49,403",2164
1171,22,077,20.3,"46,603",22077
2120,39,081,19.0,"42,821",39081
1774,31,183,14.0,"53,369",31183
2132,39,105,17.8,"44,286",39105
96,02,275,11.6,"57,832",2275
2297,42,027,18.1,"58,258",42027
2927,51,121,24.1,"52,538",51121
369,12,079,22.8,"38,249",12079


In [44]:
##Since this data frame does not have state names included, the state fips code column will be converted to 
##corresponding two-letter state name. 
##To do this, a dictionary of state fips code and state names will be generated ##from the dataframe above.

In [45]:
##isolate state fips code and name column for states only into a separate dataframe
#fips_conv=pov_inc.loc[pov_inc['County FIPS Code'] == '000']##note:county fips code =='000' for states
#fips_conv=fips_conv.filter(items=['State FIPS Code','Name'])##keep relevant columns
#fips_conv

In [46]:
##reshape data frame from long to wide 
#fips_conv = fips_conv.set_index("State FIPS Code").T

In [47]:
##view reshaped dataframe
#fips_conv

In [48]:
##convert to list dictionary
#fips_conv = fips_conv.to_dict('r')

In [49]:
## view list item
#fips_conv[0]

In [50]:
##convert list to dictionary
#fips_conv=dict(fips_conv[0])

In [51]:
##convert fips code to full state name using dictionary
#pov_inc['State FIPS Code'] = pov_inc['State FIPS Code'].apply(fips_conv.get)
##view dataframe; confirm change
#pov_inc

In [52]:
## convert state name to two-letters; use dictionary from used for NYT dataframe
#pov_inc['State FIPS Code'] = pov_inc['State FIPS Code'].apply(us_state_abbrev.get)
##view dataframe; confirm change
#pov_inc

In [53]:
## keep only relevant columns
pov_inc=pov_inc.filter(items=['fips','Poverty Percent, All Ages','Median Household Income'])
 

In [54]:
## rename column headers for consistency across data frames
pov_inc = pov_inc.rename(columns={'Poverty Percent, All Ages': 'pov_perc',
                                  'Median Household Income': 'Med_inc'})   

In [55]:
##view data frame
pov_inc

,fips,pov_perc,Med_inc
0,,13.1,"61,937"
1,1000,16.8,"49,881"
2,1001,13.8,"59,338"
3,1003,9.8,"57,588"
4,1005,30.9,"34,382"
...,...,...,...
3189,56037,8.4,"73,315"
3190,56039,6.3,"99,087"
3191,56041,10.0,"63,401"
3192,56043,11.9,"55,190"


#### Unemployment Rate
County level unemployment rate is obtained from the U.S. Bureau of Labor Statistics. The data table below represents 2019 annual average labor force data by county.  
https://www.bls.gov/lau/tables.htm

In [56]:
##import data; read FIPS code columns as dtype object to maintain traiiling zeroes.
unemp=pd.read_csv("laucnty19.xlsx - laucnty19-2.csv", dtype={'County FIPS Code': object,'State FIPSCode': object})

In [57]:
unemp

,Code,State FIPSCode,County FIPS Code,County Name/State Abbreviation,Year,Labor Force,Employed,Unemployed,Unemployment Rate
0,CN0100100000000,01,001,"Autauga County, AL",2019,"26,172","25,458",714,2.7
1,CN0100300000000,01,003,"Baldwin County, AL",2019,"97,328","94,675","2,653",2.7
2,CN0100500000000,01,005,"Barbour County, AL",2019,"8,537","8,213",324,3.8
3,CN0100700000000,01,007,"Bibb County, AL",2019,"8,685","8,419",266,3.1
4,CN0100900000000,01,009,"Blount County, AL",2019,"25,331","24,655",676,2.7
...,...,...,...,...,...,...,...,...,...
3214,CN7214500000000,72,145,"Vega Baja Municipio, PR",2019,"13,037","11,791","1,246",9.6
3215,CN7214700000000,72,147,"Vieques Municipio, PR",2019,"2,585","2,406",179,6.9
3216,CN7214900000000,72,149,"Villalba Municipio, PR",2019,"7,406","6,231","1,175",15.9
3217,CN7215100000000,72,151,"Yabucoa Municipio, PR",2019,"8,691","7,552","1,139",13.1


In [58]:
##keep only relevant columns
unemp=unemp.filter(items=['State FIPSCode','County FIPS Code','Unemployment Rate'])

In [59]:
unemp

,State FIPSCode,County FIPS Code,Unemployment Rate
0,01,001,2.7
1,01,003,2.7
2,01,005,3.8
3,01,007,3.1
4,01,009,2.7
...,...,...,...
3214,72,145,9.6
3215,72,147,6.9
3216,72,149,15.9
3217,72,151,13.1


Similar to the previous data frames, the county and state names are combined into a single column and the state and fips code are in a separate column. This will be remedied using the same tools as above

In [60]:
##join county and state fips codes
unemp['fips'] = unemp[['State FIPSCode', 'County FIPS Code']].apply(lambda x: ''.join(x), axis=1)
##remove leading zeros from fips codes
unemp['fips'] = [ i.lstrip('0') for i in unemp['fips'] ]

In [61]:
### create a separate state column
#unemp['state']=unemp['county'].str.split(",").str[1]
#unemp['state']=unemp['state'].str.strip()

In [62]:
##retain only county names in the county column
#unemp['county']= unemp['county'].str.split(",").str[0].astype(str)

In [63]:
##keep only relevant columns
unemp=unemp.filter(items=['fips','Unemployment Rate'])

In [64]:
##view dataframe
unemp 

,fips,Unemployment Rate
0,1001,2.7
1,1003,2.7
2,1005,3.8
3,1007,3.1
4,1009,2.7
...,...,...
3214,72145,9.6
3215,72147,6.9
3216,72149,15.9
3217,72151,13.1


### County characteristics: Social, phyisical, environmental, and clinical factors
The county health rankings and roadmaps program is a collaboration between the Robert Wood Johnson Foundation and the University of Wisconsin Population Health Institute. This program collects data on a range of county-level metrics such as the number of uninsured adults in a county; average traffic volume in a county, and other metrics that are used to assess the overall health of a county. Since this analysis is aimed at exploring the county-level factors that contribute to racial and ethnic disparities in COVID-19 cases, the variables in this vast dataset  can serve as vital features for the statistical learning portion. The variables selected serve as proxies for social-determinants of health that are the root-causes of health disparities.
The variables are: 
Food environment index
Access to exercise opportunities
Income inequality
Violent crime rate


https://www.countyhealthrankings.org/explore-health-rankings/rankings-data-documentation

In [65]:
##import county ranking data; read FIPS code as object to maintain leading and trailing zeroes
county_rank=pd.read_csv("analytic_data2020_0.csv", dtype={'5-digit FIPS Code': object})

/Users/mkbelay/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (0,1,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,2

In [66]:
##shape of data
county_rank.shape

(3195, 786)

In [67]:
county_rank=county_rank.filter(items=["5-digit FIPS Code","Traffic volume raw value",
                                      "Severe housing cost burden raw value","Homeownership raw value",
                                      "Residential segregation - non-White/White raw value",
                                      "Food environment index raw value",
                                      "Ratio of population to primary care physicians.",
                                      "Drinking water violations raw value",
                                      "Air pollution - particulate matter raw value","Uninsured adults raw value",
                                     "Access to exercise opportunities raw value","Income inequality raw value",
                                      "Violent crime raw value","High school graduation raw value","Some college raw value",
                                     "Social associations raw value","Percentage of households with high housing costs",
                                     "Percentage of households with overcrowding","Percentage of households with lack of kitchen or plumbing facilities",
                                     "Long commute - driving alone raw value","Life expectancy raw value","Premature age-adjusted mortality raw value",
                                     "Frequent physical distress raw value","Frequent mental distress raw value","Diabetes prevalence raw value","Food insecurity raw value",
                                     "Limited access to healthy foods raw value","% Rural raw value"])

In [68]:
county_rank

,5-digit FIPS Code,Traffic volume raw value,Severe housing cost burden raw value,Homeownership raw value,Residential segregation - non-White/White raw value,Food environment index raw value,Ratio of population to primary care physicians.,Drinking water violations raw value,Air pollution - particulate matter raw value,Uninsured adults raw value,...,Percentage of households with lack of kitchen or plumbing facilities,Long commute - driving alone raw value,Life expectancy raw value,Premature age-adjusted mortality raw value,Frequent physical distress raw value,Frequent mental distress raw value,Diabetes prevalence raw value,Food insecurity raw value,Limited access to healthy foods raw value,% Rural raw value
0,fipscode,v156_rawvalue,v154_rawvalue,v153_rawvalue,v142_rawvalue,v133_rawvalue,v004_other_data_1,v124_rawvalue,v125_rawvalue,v003_rawvalue,...,v136_other_data_3,v137_rawvalue,v147_rawvalue,v127_rawvalue,v144_rawvalue,v145_rawvalue,v060_rawvalue,v139_rawvalue,v083_rawvalue,v058_rawvalue
1,00000,NaN,0.1484650658,0.6384759732,46.77346382,7.6,1325.0905296,NaN,8.6,0.1224754716,...,NaN,0.36,79.089130214,340.42778825,0.1150841054,0.1244118215,0.103,0.125,0.0590124246,0.1926902892
2,01000,166.00847228,0.1265381674,0.6855046233,50.777775905,5.8,1542.6414557,0.1492537313,11,0.1408970991,...,NaN,0.343,75.416194148,474.78234901,0.1582685966,0.1582663382,0.142,0.163,0.0792142495,0.409631829
3,01001,88.457040416,0.1340348117,0.7489462467,23.628395199,7.2,2220.16,0,11.7,0.1114898915,...,0.0134615385,0.396,76.879477172,417.54755051,0.1368522862,0.1412934665,0.111,0.132,0.11991125,0.4200216232
4,01003,86.997429882,0.1172580795,0.7361934319,31.825343231,8,1371.7935484,0,10.3,0.1434852477,...,0.0047904192,0.417,78.450258368,353.34311052,0.1248010767,0.1347533556,0.107,0.116,0.054244266,0.4227909911
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3190,56037,154.755,0.083856,0.753702,25.3527,7.7,2720.88,1,5.1,0.169401,...,0.00786211,0.181,77.2858,384.131,0.101096,0.11,0.088,0.107,0.108453,0.109163
3191,56039,135.189,0.107691,0.577309,29.0447,8.2,861.667,1,4.9,0.175177,...,0.0262391,0.144,89.4894,127.766,0.0942373,0.103222,0.022,0.097,0.0740946,0.464309
3192,56041,96.1916,0.0770767,0.752683,11.5779,7.4,2277.22,1,5.9,0.165985,...,0.0039031,0.191,79.246,371.193,0.112189,0.119026,0.113,0.128,0.0972715,0.430959
3193,56043,82.8222,0.0681678,0.768264,10.8665,8.3,2016,0,4.8,0.193353,...,0.00858369,0.069,79.4515,346.585,0.110106,0.117185,0.117,0.112,0.0362159,0.359545


In [69]:
## remove the first row
county_rank=county_rank.iloc[1:]

In [70]:
##reset index
county_rank=county_rank.reset_index(drop=True)

In [71]:
##rename column headers
county_rank=county_rank.rename(columns={'5-digit FIPS Code':'fips',
                                       'Traffic volume raw value':'traffic_vol',
                                        'Severe housing cost burden raw value':'house_burden',
                                        'Homeownership raw value':'ownership',
                                        'Residential segregation - non-White/White raw value':'residential_seg',
                                       'Food environment index raw value':'food_index',
                                        'Ratio of population to primary care physicians.':'pop_to_phys',
                                       'Drinking water violations raw value':'water_violation',
                                        'Air pollution - particulate matter raw value':'air_pollution',
                                       'Access to exercise opportunities raw value':'exercise_opp',
                                        'Income inequality raw value':'income_ineq',
                                        'Violent crime raw value':'crime_violent', 
                                        'Uninsured adults raw value':'uninsured',
                                        "High school graduation raw value":"high_sch_grad",
                                        "Some college raw value":"some_college",
                                     "Social associations raw value":"soc_association",
                                        "Percentage of households with high housing costs":"perc_high_cost",
                                     "Percentage of households with overcrowding":"perc_overcrowding",
                                        "Percentage of households with lack of kitchen or plumbing facilities":"perc_plumb",
                                     "Long commute - driving alone raw value":"lng_commute",
                                        "Life expectancy raw value":"life_exp",
                                        "Premature age-adjusted mortality raw value":"pre_mortality",
                                     "Frequent physical distress raw value":"phys_distress",
                                        "Frequent mental distress raw value":"ment_distress",
                                        "Diabetes prevalence raw value":"diab_prevalence",
                                        "Food insecurity raw value":"food_insecure",
                                     "Limited access to healthy foods raw value":"food_access",
                                        "% Rural raw value":"perc_rural"})

In [72]:
##removing leading zero from fips column
county_rank['fips'] = [ i.lstrip('0') for i in county_rank['fips'] ]

In [73]:
##view dataframe
county_rank

,fips,traffic_vol,house_burden,ownership,residential_seg,food_index,pop_to_phys,water_violation,air_pollution,uninsured,...,perc_plumb,lng_commute,life_exp,pre_mortality,phys_distress,ment_distress,diab_prevalence,food_insecure,food_access,perc_rural
0,,NaN,0.1484650658,0.6384759732,46.77346382,7.6,1325.0905296,NaN,8.6,0.1224754716,...,NaN,0.36,79.089130214,340.42778825,0.1150841054,0.1244118215,0.103,0.125,0.0590124246,0.1926902892
1,1000,166.00847228,0.1265381674,0.6855046233,50.777775905,5.8,1542.6414557,0.1492537313,11,0.1408970991,...,NaN,0.343,75.416194148,474.78234901,0.1582685966,0.1582663382,0.142,0.163,0.0792142495,0.409631829
2,1001,88.457040416,0.1340348117,0.7489462467,23.628395199,7.2,2220.16,0,11.7,0.1114898915,...,0.0134615385,0.396,76.879477172,417.54755051,0.1368522862,0.1412934665,0.111,0.132,0.11991125,0.4200216232
3,1003,86.997429882,0.1172580795,0.7361934319,31.825343231,8,1371.7935484,0,10.3,0.1434852477,...,0.0047904192,0.417,78.450258368,353.34311052,0.1248010767,0.1347533556,0.107,0.116,0.054244266,0.4227909911
4,1005,102.29176221,0.1405954631,0.6139777923,23.449712509,5.6,3158.75,0,11.5,0.1608583414,...,0.0060307018,0.322,75.34193493,497.35006209,0.171848975,0.1658625221,0.176,0.22,0.1073966657,0.677896347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3189,56037,154.755,0.083856,0.753702,25.3527,7.7,2720.88,1,5.1,0.169401,...,0.00786211,0.181,77.2858,384.131,0.101096,0.11,0.088,0.107,0.108453,0.109163
3190,56039,135.189,0.107691,0.577309,29.0447,8.2,861.667,1,4.9,0.175177,...,0.0262391,0.144,89.4894,127.766,0.0942373,0.103222,0.022,0.097,0.0740946,0.464309
3191,56041,96.1916,0.0770767,0.752683,11.5779,7.4,2277.22,1,5.9,0.165985,...,0.0039031,0.191,79.246,371.193,0.112189,0.119026,0.113,0.128,0.0972715,0.430959
3192,56043,82.8222,0.0681678,0.768264,10.8665,8.3,2016,0,4.8,0.193353,...,0.00858369,0.069,79.4515,346.585,0.110106,0.117185,0.117,0.112,0.0362159,0.359545


## Merging
Now that the data frames have been cleaned and column headers (specifically for fips and county) have been standardized, the data frame will be combined into one with the NYT data set serving as the main/basis data frame.

####  NYT + Population (master)

In [74]:
##master dataframe will be called df; start with merging NYT dataframe and population dataframe
df=pd.merge(left=county_data, right=county_pop, how='left',on=["fips"], indicator=True)

In [75]:
##view left-only merges; 
df.loc[df._merge=="left_only",:].drop(columns= "_merge")

,fips,state,county,cases,white_cases,black_cases,hispanic_cases,native_cases,asian_cases,TOT_POP,WH,BA,AA,IA,HI


All observations in the NYT COVID-19 dataframe have been merged with their corresponding population data in the county population dataframe.

In [76]:
##drop _merge column
df=df.drop(columns= "_merge")

In [77]:
##view dataframe
df.sample(10)

,fips,state,county,cases,white_cases,black_cases,hispanic_cases,native_cases,asian_cases,TOT_POP,WH,BA,AA,IA,HI
233,13313,GA,Whitfield County,235,17,0,92,0,0,104628,94781,4607,1715,1492,38014
680,37093,NC,Hoke County,211,12,29,10,0,0,55234,27081,19594,817,5018,7695
336,19191,IA,Winneshiek County,15,5,0,0,0,0,19991,19373,170,240,30,474
106,8029,CO,Delta County,27,22,0,0,0,0,31162,29546,270,269,425,4769
281,17157,IL,Randolph County,235,131,0,50,0,0,31782,27697,3371,163,110,1054
758,39137,OH,Putnam County,15,5,0,0,0,0,33861,33170,211,91,119,2220
57,2090,AK,Fairbanks North Star Borough,65,27,0,0,0,0,96849,73230,5148,3283,7916,7938
406,25023,MA,Plymouth County,8022,2508,1519,418,0,30,521202,438775,61019,8442,1770,21739
666,37061,NC,Duplin County,711,128,160,359,0,0,58741,41353,14962,440,804,13517
475,27067,MN,Kandiyohi County,457,61,74,84,0,0,43199,38934,2908,521,236,5444


### Master +GDP

In [78]:
## merge master dataframe from above with GDP data
df = pd.merge(left=df, right=gdp, how='left',on=["fips"], indicator=True)

In [79]:
##view left-only merges; 
df.loc[df._merge=="left_only",:]

,fips,state,county,cases,white_cases,black_cases,hispanic_cases,native_cases,asian_cases,TOT_POP,WH,BA,AA,IA,HI,GDP,_merge
873,51003,VA,Albemarle County,166,76,10,0,0,0,109330,89388,10600,6051,383,6313,NaN,left_only
875,51015,VA,Augusta County,80,53,0,0,0,0,75558,70102,3518,490,191,2374,NaN,left_only
879,51035,VA,Carroll County,70,38,0,11,0,0,29791,28964,263,68,98,1147,NaN,left_only
882,51059,VA,Fairfax County,10209,1308,569,5073,0,722,1147532,742178,121954,231085,6108,189515,NaN,left_only
886,51069,VA,Frederick County,292,121,0,47,0,0,89313,80867,4225,1573,431,8325,NaN,left_only
888,51081,VA,Greensville County,54,0,10,0,0,0,11336,4281,6823,79,32,316,NaN,left_only
891,51089,VA,Henry County,80,11,5,0,0,0,50557,37625,11434,254,180,2926,NaN,left_only
892,51095,VA,James City County,179,113,0,0,0,0,76523,61473,10433,2023,336,4692,NaN,left_only
897,51121,VA,Montgomery County,51,11,0,0,0,0,98535,85207,4238,6245,307,3424,NaN,left_only
901,51143,VA,Pittsylvania County,28,6,0,0,0,0,60354,45990,12970,303,199,1663,NaN,left_only


About 20 counties in VA seem to be missing from the GDP data. Let's query these counties in the GDP using the fips code to see if it is a naming issue or if these observations are not present in the GDP dataframe

In [80]:
## isolate left_only merges
merge_fail_gdp=df.loc[df._merge=="left_only",:]

In [81]:
##turn column of fips to list
merge_fail_gdp=merge_fail_gdp["fips"]. tolist() 
merge_fail_gdp

['51003',
 '51015',
 '51035',
 '51059',
 '51069',
 '51081',
 '51089',
 '51095',
 '51121',
 '51143',
 '51149',
 '51153',
 '51161',
 '51165',
 '51175',
 '51177',
 '51191',
 '51199',
 '51775']

In [82]:
##check if the fips code of observations in the left-only merge are in the GDP data
gdp[gdp['fips'].isin(merge_fail_gdp)]

,fips,GDP


Looks like these observations are not in the GDP dataframe. Let's do a spot check to make sure

In [83]:
##fips code that did not merge
merge_fail_gdp

['51003',
 '51015',
 '51035',
 '51059',
 '51069',
 '51081',
 '51089',
 '51095',
 '51121',
 '51143',
 '51149',
 '51153',
 '51161',
 '51165',
 '51175',
 '51177',
 '51191',
 '51199',
 '51775']

In [84]:
##create a list to house random sample of merge failed fip codes
merge_fail_samp=sample(merge_fail_gdp,5)

In [85]:
##view random sample of fips codes
merge_fail_samp

['51095', '51161', '51153', '51143', '51003']

In [86]:
##check if random sample of fips codes are in gdp dataframe
gdp.loc[gdp['fips'].isin(merge_fail_samp)]

,fips,GDP


Since these fips codes are not in the GDP data, the _merge column will be dropped and the resot of the dataframe will be merged

In [87]:
df=df.drop(columns= "_merge")

### Master + Poverty & Median Household Income

In [88]:
## merge master dataframe from above with poverty and median household income data
df = pd.merge(left=df, right=pov_inc, how='left',on=["fips"], indicator=True)

In [89]:
##view left-only merges; 
df.loc[df._merge=="left_only",:]

,fips,state,county,cases,white_cases,black_cases,hispanic_cases,native_cases,asian_cases,TOT_POP,WH,BA,AA,IA,HI,GDP,pov_perc,Med_inc,_merge


Merge was successful.

In [90]:
##drop _merge column
df=df.drop(columns= "_merge")

### Master + Unemployment Rate

In [91]:
## merge master dataframe from above with Unemployment data
df=pd.merge(left=df, right=unemp, how='left',on=['fips'], indicator=True)

In [92]:
##view left-only merges; 
df.loc[df._merge=="left_only",:]

,fips,state,county,cases,white_cases,black_cases,hispanic_cases,native_cases,asian_cases,TOT_POP,WH,BA,AA,IA,HI,GDP,pov_perc,Med_inc,Unemployment Rate,_merge


Merge was successful.

In [93]:
##drop _merge column
df=df.drop(columns= "_merge")

### Master + County Rank Metrics

In [94]:
## merge master dataframe from above with county rankings data
df = pd.merge(left=df, right=county_rank, how='left',on=["fips"], indicator=True)

In [95]:
##view left-only merges; 
df.loc[df._merge=="left_only",:]

,fips,state,county,cases,white_cases,black_cases,hispanic_cases,native_cases,asian_cases,TOT_POP,...,lng_commute,life_exp,pre_mortality,phys_distress,ment_distress,diab_prevalence,food_insecure,food_access,perc_rural,_merge


Merge was successful.

In [96]:
##drop _merge column
df=df.drop(columns= "_merge")

In [97]:
## view final dataframe
df.sample(20)

,fips,state,county,cases,white_cases,black_cases,hispanic_cases,native_cases,asian_cases,TOT_POP,...,perc_plumb,lng_commute,life_exp,pre_mortality,phys_distress,ment_distress,diab_prevalence,food_insecure,food_access,perc_rural
571,30101,MT,Toole County,8,8,0,0,0,0,4736,...,0.0880829,0.195,76.6638,461.249,0.10926,0.117244,0.052,0.129,0.11768,0.429564
167,12099,FL,Palm Beach County,4026,1220,742,1122,0,0,1496770,...,0.00864358,0.364,82.286989505,296.77711684,0.1231761523,0.1316516098,0.099,0.129,0.046543735,0.0104254568
317,19113,IA,Linn County,928,536,220,18,0,16,226706,...,0.0088063178,0.15,80.221422816,296.07468468,0.0863068332,0.1062281162,0.093,0.108,0.0460709741,0.1274795716
579,34013,NJ,Essex County,17073,1934,4784,2806,6,226,798975,...,0.0124696,0.446,79.4998,347.847,0.11726,0.122629,0.106,0.163,0.00706099,0.000145414
191,13067,GA,Cobb County,1414,229,130,16,0,0,760141,...,0.0058645096,0.534,80.521249362,268.57235992,0.0925574999,0.1051493617,0.087,0.111,0.1022719365,0.0024619302
911,51179,VA,Stafford County,624,101,43,181,0,0,152882,...,0.00733469,0.545,80.3237,270.944,0.0890782,0.099896,0.073,0.063,0.0343126,0.197548
273,17119,IL,Madison County,505,243,97,0,0,0,262966,...,0.010512545,0.377,76.74025434,409.38269433,0.1024866297,0.1166902725,0.108,0.116,0.1206606976,0.1334066146
568,28161,MS,Yalobusha County,71,6,28,0,0,0,12108,...,0.0058309,0.357,73.2123,565.627,0.148229,0.148278,0.153,0.191,0.0396992,0.793185
360,20177,KS,Shawnee County,281,114,5,66,0,0,176875,...,0.0119693023,0.127,76.905978289,414.20706528,0.1053230324,0.1136453933,0.113,0.133,0.0949359989,0.158244068
562,28149,MS,Warren County,131,40,38,0,0,0,45381,...,0.00866035,0.176,73.5756,556.338,0.146564,0.143391,0.184,0.215,0.112726,0.408935


### Export complete dataframe to csv file
The data collection and high-level cleaning process is complete. The merged dataframe will be exported as a csv file to be used for visualizations and modeling. 

In [98]:
##check current working directory
pwd

NameError: name 'pwd' is not defined

In [99]:
##export as csv file
df.to_csv("COVID_project_PPOL564_v2.csv", index=False)